In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import csv
import os
import sys
import time

In [2]:
driver = webdriver.Chrome('./chromedriver')

### 幣種選擇
* 人民幣(CNY)
* 美元(USD)
* 澳幣(AUD)
* 南非幣(ZAR)
* 紐西蘭幣(NZD)
* 歐元(EUR)
* 港幣(HKD)
* 日圓(JPY)
* 墨西哥披索(MXN)
* 加幣(CAD)
* 瑞士法郎(CHF)
* 英鎊(GBP)
* 瑞典幣(SEK)
* 新加坡幣(SGD)
* 泰銖(THB)

In [3]:
currency = str.upper(input())

USD


### 即期或現鈔

* 即期輸入:spot
* 現鈔輸入:cash


In [4]:
rate_type = str.lower(input())

SPOT


### 時間

* 當日輸入:Now
* 3個月輸入:3Months
* 6個月輸入:6Months
* 1年輸入:1Year


In [5]:
duration = str.upper(input())

1YEAR


In [6]:
if rate_type == "spot":
     type_html ="/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[1]/div/img[2]"
elif rate_type == "cash":
     type_html ="/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[1]/div/img[1]"

In [7]:
if duration == "NOW":
     duration_html ="/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[2]/div[1]/img[1]"
elif duration == "3MONTHS":
     duration_html ="/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[2]/div[1]/img[2]"
elif duration == "6MONTHS":
     duration_html ="/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[2]/div[1]/img[3]"
elif duration == "1YEAR":
     duration_html ="/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[2]/div[1]/img[4]"

In [8]:
# Data page
driver.get("https://www.esunbank.com.tw/bank/personal/deposit/rate/forex/exchange-rate-chart")

time.sleep(1)

In [9]:
# Select the currency 
select_currency_arrow = driver.find_element_by_xpath("/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[1]/ul/li/span")
select_currency_arrow.click()

# Select the currency
select_currency = driver.find_elements_by_css_selector(".transformSelectDropdown li span")
for i in select_currency:
    if currency in i.text:
        i.click()
        break

time.sleep(1)

In [10]:
# Select Spot or Cash
currency_type = driver.find_element_by_xpath(type_html)
currency_type.click()

time.sleep(1)

In [11]:
# Select the duration
currency_duration = driver.find_element_by_xpath(duration_html)
currency_duration.click()

time.sleep(1)

In [12]:
# Select data type
data_type = driver.find_element_by_xpath("/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[3]/div/label[2]")
data_type.click()

time.sleep(1)

In [13]:
# Click search button
search_button = driver.find_element_by_xpath('/html/body/form/div[7]/div[3]/div[4]/div/div[1]/div[4]/a')
search_button.click()

time.sleep(1)

In [14]:
df = pd.DataFrame(columns=["Date", "Bid_Rate", "Offer_Rate"])
pages = True

while pages:
    items = driver.find_elements_by_css_selector("#inteTable tbody tr")

    for item in items:
        tds = item.find_elements_by_css_selector("td")

        if tds[0].get_attribute("class") != "itemTtitle":
            continue

        df = df.append({
                    "Date": tds[0].text,
                    "Bid_Rate":  tds[1].text,
                    "Offer_Rate":  tds[2].text
                    }, ignore_index=True)

    nextBtn = driver.find_element_by_css_selector(".pageNumberBlock .down")
    if "active" in nextBtn.get_attribute("class"):
        nextBtn.click()
    else:
        pages = False

time.sleep(4)

In [15]:
# Close the browser
driver.quit()

In [16]:
df

,Date,Bid_Rate,Offer_Rate
0,2019-04-17,30.8,30.9
1,2019-04-18,30.79,30.89
2,2019-04-19,30.78,30.88
3,2019-04-22,30.81,30.91
4,2019-04-23,30.81,30.91
...,...,...,...
245,2020-04-10,30.03,30.13
246,2020-04-13,30.02,30.12
247,2020-04-14,29.98,30.08
248,2020-04-15,30,30.1


In [17]:
df[df.Offer_Rate == df.Offer_Rate.min()]

,Date,Bid_Rate,Offer_Rate
189,2020-01-14,29.85,29.95
